In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from itertools import product
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import itertools
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_curve, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import statsmodels.api as sm
%matplotlib inline
# Ignore warnings
warnings.filterwarnings("ignore")

In [51]:
playstore = pd.read_csv('googleplaystore.csv')

In [52]:
playstore.shape

(10841, 13)

In [53]:
playstore.drop_duplicates(inplace=True)
playstore.shape

(10358, 13)

In [54]:
playstore

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [47]:
playstore.describe(include='object')

,App,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
count,10358,10358,10358,10358,10358,10357,10358,10357,10358,10358,10350,10355
unique,9660,34,6002,462,22,3,93,6,120,1378,2832,33
top,ROBLOX,FAMILY,0,Varies with device,"1,000,000+",Free,0,Everyone,Tools,"August 3, 2018",Varies with device,4.1 and up
freq,9,1943,593,1526,1488,9591,9592,8382,842,298,1302,2379


In [48]:
import pandas as pd
import numpy as np

def version_key(version):
    if pd.isna(version) or version == 'Varies with device':
        return float('inf')
    
    parts = version.replace(' and up', '').replace(' - ', '.').split('.')
    major = int(parts[0]) if parts[0].isdigit() else 0
    minor = int(parts[1]) if len(parts) > 1 and parts[1].isdigit() else 0
    patch = int(parts[2]) if len(parts) > 2 and parts[2].isdigit() else 0
    return major, minor, patch

# Convert the 'data' column to string type to handle NaN values
playstore['Android Ver'] = playstore['Android Ver'].astype(str)

# Filter out unwanted versions
allowed_versions = ['4.0.3 and up', '4.2 and up', '4.4 and up', '2.3 and up', '3.0 and up', '4.1 and up', '4.0 and up', '2.3.3 and up', 'Varies with device', '2.2 and up', '5.0 and up', '6.0 and up', '1.6 and up', '1.5 and up', '2.1 and up', '7.0 and up', '5.1 and up', '4.3 and up', '4.0.3 - 7.1.1', '2.0 and up', '3.2 and up', '4.4W and up', '7.1 and up', '7.0 - 7.1.1', '8.0 and up', '5.0 - 8.0', '3.1 and up', '2.0.1 and up', '4.1 - 7.1.1', '5.0 - 6.0', '1.0 and up', '2.2 - 7.1.1', '5.0 - 7.1.1']
playstore = playstore[playstore['Android Ver'].isin(allowed_versions)]

# Extract the major versions
playstore['major_version'] = playstore['Android Ver'].apply(lambda x: x.split(' and up')[0].split(' - ')[0].split('.')[0])


In [49]:
# content_rating
def fix_content_rating(content_rating):
    if content_rating == 'Teen' and 'Everyone 10+':
        return 'Restricted'
    elif content_rating == 'Mature 17+':
        return 'Restricted'
    elif content_rating == 'Adults only 18+':
        return 'Restricted'
    else:
        return 'Everyone'
    
playstore['Content Rating'] = playstore['Content Rating'].apply(lambda x: fix_content_rating(x))
playstore['Content Rating']

0          Everyone
1          Everyone
2          Everyone
3        Restricted
4          Everyone
            ...    
10836      Everyone
10837      Everyone
10838      Everyone
10839    Restricted
10840      Everyone
Name: Content Rating, Length: 10355, dtype: object

In [50]:
playstore['Content Rating'].value_counts()

Everyone      8759
Restricted    1596
Name: Content Rating, dtype: int64

In [42]:
playstore['Reviews'] = pd.to_numeric(playstore['Reviews'], errors='coerce')

In [9]:
# Function to remove the plus sign and convert to integer
def remove_plus_and_convert_to_int(value):
    # Handle 'Free' as 0
    if value == 'Free':
        return 0
    else:
        # Use regex to remove '+' and ',' characters and convert to integer
        value = re.sub(r'[+,]', '', value)
        return int(value)

# Import the regex module
import re

# Convert the 'Installs' column to integers
playstore['Installs'] = playstore['Installs'].map(remove_plus_and_convert_to_int)

In [10]:
playstore['Installs']

0           10000
1          500000
2         5000000
3        50000000
4          100000
           ...   
10836        5000
10837         100
10838        1000
10839        1000
10840    10000000
Name: Installs, Length: 10358, dtype: int64

In [11]:
def convert_to_kilobytes(value):
    match = re.match(r'^(-?\d+(\.\d+)?)\s*([mk])?$', value, re.IGNORECASE)
    if match:
        numeric_value = float(match.group(1))
        unit = match.group(3)
        if unit:
            unit = unit.lower()
            if unit == 'm':
                return int(numeric_value * 1024)
            elif unit == 'k':
                return int(numeric_value)
        else:
            return int(numeric_value)
    return None

# Apply the conversion function to the DataFrame column
playstore['Size'] = playstore['Size'].apply(convert_to_kilobytes)

In [12]:
playstore['Size']

0        19456.0
1        14336.0
2         8908.0
3        25600.0
4         2867.0
          ...   
10836    54272.0
10837     3686.0
10838     9728.0
10839        NaN
10840    19456.0
Name: Size, Length: 10358, dtype: float64

In [13]:
playstore['Price'].unique()

array(['0', '$4.99', '$3.99', '$6.99', '$1.49', '$2.99', '$7.99', '$5.99',
       '$3.49', '$1.99', '$9.99', '$7.49', '$0.99', '$9.00', '$5.49',
       '$10.00', '$24.99', '$11.99', '$79.99', '$16.99', '$14.99',
       '$1.00', '$29.99', '$12.99', '$2.49', '$10.99', '$1.50', '$19.99',
       '$15.99', '$33.99', '$74.99', '$39.99', '$3.95', '$4.49', '$1.70',
       '$8.99', '$2.00', '$3.88', '$25.99', '$399.99', '$17.99',
       '$400.00', '$3.02', '$1.76', '$4.84', '$4.77', '$1.61', '$2.50',
       '$1.59', '$6.49', '$1.29', '$5.00', '$13.99', '$299.99', '$379.99',
       '$37.99', '$18.99', '$389.99', '$19.90', '$8.49', '$1.75',
       '$14.00', '$4.85', '$46.99', '$109.99', '$154.99', '$3.08',
       '$2.59', '$4.80', '$1.96', '$19.40', '$3.90', '$4.59', '$15.46',
       '$3.04', '$4.29', '$2.60', '$3.28', '$4.60', '$28.99', '$2.95',
       '$2.90', '$1.97', '$200.00', '$89.99', '$2.56', '$30.99', '$3.61',
       '$394.99', '$1.26', 'Everyone', '$1.20', '$1.04'], dtype=object)

In [14]:
# Function to convert the price to integers
def convert_price_to_int(price):
    if price == 'Everyone':
        return 0
    else:
        return int(float(''.join(filter(str.isdigit, price))))

# Apply the conversion function to the 'price' column
playstore['Price'] = playstore['Price'].apply(convert_price_to_int)

In [15]:
playstore['Price']

0        0
1        0
2        0
3        0
4        0
        ..
10836    0
10837    0
10838    0
10839    0
10840    0
Name: Price, Length: 10358, dtype: int64

In [16]:
print(playstore['Rating'].describe(), playstore['Size'].describe())

count    8893.000000
mean        4.189542
std         0.545452
min         1.000000
25%         4.000000
50%         4.300000
75%         4.500000
max        19.000000
Name: Rating, dtype: float64 count      8831.000000
mean      21798.151625
std       23081.680805
min           8.000000
25%        4812.000000
50%       13312.000000
75%       29696.000000
max      102400.000000
Name: Size, dtype: float64


In [17]:
from sklearn.impute import KNNImputer
numerical_columns = playstore.select_dtypes(include='number').columns

# Check for missing values
print(playstore.isnull().sum())
numerical_columns = playstore.select_dtypes(include='number').columns

# KNN imputation for numerical data
knn_imputer_num = KNNImputer(n_neighbors=5, weights='uniform')
playstore[numerical_columns] = knn_imputer_num.fit_transform(playstore[numerical_columns])


App                  0
Category             0
Rating            1465
Reviews              1
Size              1527
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64


In [18]:
print(playstore.isnull().sum())

App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              1
Price             0
Content Rating    1
Genres            0
Last Updated      0
Current Ver       8
Android Ver       3
dtype: int64


In [19]:
print(playstore['Rating'].describe(), playstore['Size'].describe())

count    10358.000000
mean         4.288417
std          0.759210
min          1.000000
25%          4.000000
50%          4.300000
75%          4.500000
max         19.000000
Name: Rating, dtype: float64 count     10358.000000
mean      23237.339139
std       22605.621810
min           8.000000
25%        5734.000000
50%       15360.000000
75%       33792.000000
max      102400.000000
Name: Size, dtype: float64
